Note that the variables in the houshold problem e, a, are in terms of non-tradable goods. We can convert them to tradable goods by multiplying them by the price of the non-tradable good. Consumption outputs c are in real terms.

In [4]:
%load_ext autoreload
%autoreload 2


import numpy as np
import sys

import matplotlib.pyplot as plt   
import seaborn as sns


colors = plt.rcParams['axes.prop_cycle'].by_key()['color']



plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

# # Apply style settings
# plt.rcParams.update({
#     'font.family': 'serif',
#     'font.serif': ['Palatino'],
#     'text.usetex': True
# })

# # Use seaborn style
# sns.set_style('white')


# from scipy.interpolate import CubicSpline
# from scipy.stats import norm

# import matplotlib.pyplot as plt   
# from matplotlib.ticker import FormatStrFormatter

# from seaborn import set_palette
# from matplotlib import rc
# plt.style.use('seaborn-v0_8-white')
# set_palette("colorblind")
# rc('font',**{'family':'serif','serif': ['Palatino']})
# rc('text',usetex=True)

from IHANKModel import IHANKModelClass
import figures as figs
import calculations

run_test = False
save_figs = False
path = '/Users/emmagundersen/Desktop/Current code/I_HANK_PIGL1703/fig/1903/'
lvl_path = ['U_hh', 'Walras', 'CB', 'i', 'NX', 'CA', 'ra', 'iF_s',  'iF_s']
path_philips= ['WTH', 'WNT', 'NTH', 'NNT', 'PTH', 'PNT', 'PF', 'UC_TH_hh', 'UC_NT_hh', 'EX']
paths = ['PE_s', 'PE', 'YH',  'W', 'P', 'w', 'tau',  'N', 'INC', 'inc',  'CT', 'CNT', 'p', 'E', 'PF_s', 'PF', 'CTH_s', 'i', 'ra', 'iF_s']
target_var = ['PE_s', 'PE', 'p', 'inc_NT', 'inc_TH', 'ra', 'i', 'Q']



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

# # Basline preferences
# model = IHANKModelClass(name='baseline')

# model.par.eta_s = 0.51
# model.par.etaF = 0.51
# model.par.etaE = 0.1
# model.par.float = True
# model.par.pf_fixed = False
# model.par.mon_policy = 'real'
# model.par.phi_inflation = 1.5

# model.find_ss(do_print=False)
# model.compute_jacs()

# shock_PE_s, shock_forigne_interest, shock_PF_s, shock_PF_s_taylor, shock_PE_PF, shock_PE_PF_taylor, shock_i = calculations.difine_shocks(model, scale=1.0, rho = 0.8) 

In [5]:
paths_ = ['PE', 'PT', 'p', 'CT', 'CNT', 'YH', 'P', 'W', 'w']

In [6]:
model = IHANKModelClass(name='baseline')


In [15]:
model = IHANKModelClass(name='baseline')

model.par.eta_s = 0.5
model.par.etaF = 0.5
model.par.etaE = 0.1
# model.par.gamma_ = 
# model.par.tau_ss = 0.1
# model.par.gamma = model.par.epsilon
# model.par.epsilon_ = 0.0 # for homothetic preferences
model.par.float = True
model.par.pf_fixed = False
model.par.mon_policy = 'taylor'
model.par.phi = 1.5
# model.par.kappa = .2
model.par.mu_p = 1.2


model.find_ss(do_print=False)
# model.compute_jacs()

shock_PE_i, shock_PE_s, shock_forigne_interest, shock_PF_s, shock_PF_s_taylor, shock_PE_PF, shock_PE_PF_taylor, shock_i = calculations.difine_shocks(model, scale=0.03, rho = 0.8) 



nu = 0.70
sT = 0.50


nu = 0.70
sT = 0.50


nu = 0.70
sT = 0.50


nu = 0.70
sT = 0.50


nu = 0.70
sT = 0.50


nu = 58.43
sT = 69.34


nu = 35.32
sT = 26.24


nu = 0.70
sT = 0.50


nu = 0.70
sT = 0.50


nu = 58.43
sT = 69.34


nu = 35.32
sT = 26.24


nu = 18.54
sT = 12.29


nu = 14.14
sT = 9.18


nu = 49.89
sT = 32.60


nu = 32.09
sT = 20.74


nu = 14.14
sT = 9.18


nu = 14.14
sT = 9.18


nu = 23.12
sT = 14.95


nu = 32.10
sT = 20.70


nu = 50.08
sT = 32.20


nu = 86.02
sT = 55.20


nu = 157.92
sT = 101.20


nu = 301.71
sT = 193.19


nu = 589.29
sT = 377.18


nu = 1164.46
sT = 745.15


nu = 2314.80
sT = 1481.10


nu = 4615.47
sT = 2952.99


nu = 9216.81
sT = 5896.78


nu = 18419.50
sT = 11784.36


nu = 36824.87
sT = 23559.51


nu = 73635.61
sT = 47109.81
Share of domestic workers in tradable sector = 47109.81


nu = 73635.61
sT = 47109.81
Average share of consumption of tradablespar.omega_T = 73635.610
Average elasticity of substitution between tradable and non-tradable goodspar.eta_T

In [16]:
model.test_ss()

E              :       1.0000
CB             :       1.0000
ZTH            :       1.0000
ZNT            :       1.0000
NTH            :   47109.8092
NNT            :  -47108.8092
piWTH          :       0.0000
piWNT          :       0.0000
YTH            :   47109.8092
YNT            :  -47108.8092
WTH            :       0.8333
WNT            :       0.8333
PTH            :       1.0000
PNT            :       1.0000
pi_TH          :       0.0000
pi_NT          :       0.0000
PF_s           :       1.0000
PF             :       1.0000
PTH_s          :       1.0000
PT             :       1.0000
P              :       1.0000
pi_F_s         :       0.0000
pi_F           :       0.0000
pi_T           :       0.0000
pi             :       0.0000
pi_TH_s        :       0.0000
PE             :       1.0000
PTHF           :       1.0000
PE_s           :       1.0000
Q              :       1.0000
i              :       0.0050
i_shock        :       0.0000
r_real         :       0.0050
G         

In [ ]:
# model.ss.inc_TH = 1.0
# model.ss.inc_NT = 1.0
# # model.ss.p = 1.0
# model.ss.ra = 0.01
model.find_ss()
# model.solve_hh_ss()
# print(f'inc_TH = {model.ss.inc_TH}')
# print(f'inc_NT = {model.ss.inc_NT}')
# print(f'p = {model.ss.p}')
# print(f'ra = {model.ss.ra}')


# print(f'A_hh = {model.ss.A_hh}')
# print(f'CT = {model.ss.CT_hh}')
# print(f'CNT = {model.ss.CNT_hh}')
# print(f'E = {model.ss.E_hh}')


In [ ]:
figs.plot_policy(model, ['a', 'e', 'ct', 'cnt'])

In [ ]:
model.test_ss()

In [ ]:
model.compute_jacs()

In [ ]:
model.test_path()

In [ ]:
model_baseline = model.copy(name='Floating exchange rate')
# model_baseline.par.gamma = 0.13
model_baseline.find_transition_path(shocks=shock_PE_s, do_end_check=False)

# model_baseline.show_IRFs(target_var, T_max=500, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)
# model_baseline.show_IRFs(paths, T_max=500, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
model_baseline.show_IRFs(model.varlist, T_max=50, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
model_homothetic = model.copy(name='Homothetic preferences')
model_homothetic.par.epsilon = 0.0000
model_homothetic.par.gamma = 0.1
model_homothetic.find_ss(do_print=False)
model_homothetic.compute_jacs()
model_homothetic.find_transition_path(shocks=shock_PE_s, do_end_check=False)


In [ ]:

model_homothetic.solve_hh_ss()

In [ ]:
model_homothetic.compute_jacs()

In [ ]:
model_homothetic.find_transition_path(shocks=shock_PE_s, do_end_check=False)


In [ ]:
model.compare_IRFs([model_baseline, model_homothetic], ['Baseline', 'Homothetic Preferens'] target_var, T_max=500, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
v_sum = np.sum(model_baseline.ss.v)
v_sum

In [ ]:
np.sum(model.ss.z_trans[0,0,:])

In [ ]:
v_sum = np.sum(model_baseline.path.v[:,:,:,0])
v_sum

In [ ]:
model_baseline.show_IRFs(paths_, T_max=50, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
model_baseline.show_IRFs(['YTH', 'YNT', 'CTH', 'CTH_s', 'PTH_s'], T_max=50, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
paths_2 = ['P', 'i', 'r_real', 'E', 'Q', 'PF']
model_baseline.show_IRFs(paths_2, T_max=50, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
model

In [ ]:
plt.plot(model_baseline.path.B/model_baseline.path.PNT)
plt.hlines(model_baseline.ss.B, 0, 500)

In [ ]:
model.show_IRFs(target_var, T_max=500, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
gamma_list = np.linspace(0.1,0.18, 10 )
eta_list = []
for gamma in gamma_list: 
    model.par.gamma = gamma
    model.find_ss()
    eta = model.par.eta_T_RA
    eta_list.append(eta)
    
plt.plot(gamma_list, eta_list)

In [ ]:
model_PIGL_cubdouglaseta = model.copy(name='PIGL preferences with cubic douglas eta')
model_PIGL_cubdouglaseta.par.gamma = 0.13
model_PIGL_cubdouglaseta.find_ss()
model_PIGL_cubdouglaseta.compute_jacs()
model_PIGL_cubdouglaseta.find_transition_path(shocks=shock_PE_s, do_end_check=False)

In [ ]:
model_CB = model.copy(name='Cubb Douglas preferences')
model_CB.par.pref = 'CUBB_douglas'
model_CB.find_ss(do_print=False)
model_CB.compute_jacs()
model_CB.find_transition_path(shocks=shock_PE_s, do_end_check=False)

In [ ]:
Baseling_fig = figs.plot_jac_p(model_baseline, title='Baseline')
Baseling_fig.savefig(path + 'jac_p_baseline.png')
CB_fig = figs.plot_jac_p(model_CB, title='Cobb-Douglas preferences')
CB_fig.savefig(path + 'jac_p_CB.png')
PIGL_fig = figs.plot_jac_p(model_PIGL_cubdouglaseta, title='PIGL preferences with avergae eta= 1')
PIGL_fig.savefig(path + 'jac_p_PIGL.png')

In [ ]:
# Ploting all jacs
def plot_jac(model, title = None):

   fig = plt.figure(figsize=(15, 15))
   # tittle
   if title != None:
      fig.suptitle(title)

   ax = fig.add_subplot(3,3,3)
   ax.set_title(r'Expenditure wrt $\tilde p$')
   for s in [0, 50, 150, 250]:
      jac_hh_var = model.jac_hh[('E_hh', 'p')]
      ax.plot(np.arange(model.par.T), jac_hh_var[:, s],  label=f'shock at {s}')
   ax.set_ylabel(r'$dC_{NT}$')
   ax.set_xlabel('Quarters')

   ax = fig.add_subplot(3,3,1)
   ax.set_title(r'Consumption of tradables wrt $\tilde p$')
   for s in [0, 50, 150, 250]:
      jac_hh_var = model.jac_hh[('CT_hh', 'p')]
      ax.plot(np.arange(model.par.T), jac_hh_var[:, s],  label=f'shock at {s}')
   ax.set_ylabel(r'$dC_T$')

   ax = fig.add_subplot(3,3,2)
   ax.set_title(r'Consumption of Non-tradables wrt $\tilde p$')
   for s in [0, 50, 150, 250]:
      jac_hh_var = model.jac_hh[('CNT_hh', 'p')]
      ax.plot(np.arange(model.par.T), jac_hh_var[:, s],  label=f'shock at {s}')
   ax.set_ylabel(r'$dC_{NT}$')
   ax.set_xlabel('Quarters')



   ax = fig.add_subplot(3,3,4)
   ax.set_title(r'Expenditure wrt $\tilde w_NT$')
   for s in [0, 50, 150, 250]:
      jac_hh_var = model.jac_hh[('E_hh', 'inc_TH')]
      ax.plot(np.arange(model.par.T), jac_hh_var[:, s],  label=f'shock at {s}')
   ax.set_ylabel(r'$dC_{NT}$')
   ax.set_xlabel('Quarters')

   ax = fig.add_subplot(3,3,5)
   ax.set_title(r'Consumption of tradables wrt $w_NT$')
   for s in [0, 50, 150, 250]:
      jac_hh_var = model.jac_hh[('CT_hh', 'inc_TH')]
      ax.plot(np.arange(model.par.T), jac_hh_var[:, s],  label=f'shock at {s}')
   ax.set_ylabel(r'$dC_T$')

   ax = fig.add_subplot(3,3,6)
   ax.set_title(r'Consumption of Non-tradables wrt $w_NT$')
   for s in [0, 50, 150, 250]:
      jac_hh_var = model.jac_hh[('CNT_hh', 'inc_TH')]
      ax.plot(np.arange(model.par.T), jac_hh_var[:, s],  label=f'shock at {s}')
   ax.set_ylabel(r'$dC_{NT}$')
   ax.set_xlabel('Quarters')




   ax = fig.add_subplot(3,3,7)
   ax.set_title(r'Expenditure wrt $\tilde r$')
   for s in [0, 50, 150, 250]:
      jac_hh_var = model.jac_hh[('E_hh', 'ra')]
      ax.plot(np.arange(model.par.T), jac_hh_var[:, s],  label=f'shock at {s}')
   ax.set_ylabel(r'$dC_{NT}$')
   ax.set_xlabel('Quarters')

   ax = fig.add_subplot(3,3,8)
   ax.set_title(r'Consumption of tradables wrt $\tilde r$')
   for s in [0, 50, 150, 250]:
      jac_hh_var = model.jac_hh[('CT_hh', 'ra')]
      ax.plot(np.arange(model.par.T), jac_hh_var[:, s],  label=f'shock at {s}')
   ax.set_ylabel(r'$dC_T$')

   ax = fig.add_subplot(3,3,9)
   ax.set_title(r'Consumption of Non-tradables wrt $\tilde r$')
   for s in [0, 50, 150, 250]:
      jac_hh_var = model.jac_hh[('CNT_hh', 'ra')]
      ax.plot(np.arange(model.par.T), jac_hh_var[:, s],  label=f'shock at {s}')
   ax.set_ylabel(r'$dC_{NT}$')
   ax.set_xlabel('Quarters')



   # legende outside box
#    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
   plt.legend
   plt.tight_layout()

   return fig

fig = plot_jac(model_baseline, title='Baseline')
fig.savefig(path + 'jac_all.png')

In [ ]:
model_baseline.compare_IRFs([model_baseline, model_CB, model_PIGL_cubdouglaseta], ['Baseline', 'Cubb douglas', 'PIGL with eta = 1.0'],paths,  T_max=500, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
model_CES = model.copy(name='CES preferences')
model_CES.par.pref = 'CES'
model_CES.par.gamma = model_CES.par.epsilon 
model_CES.find_ss()
model_CES.compute_jacs()
model_CES.find_transition_path(shocks=shock_PE_s, do_end_check=False)

In [ ]:
model_PIGL_compar = model.copy(name='Floating exchange rate')
model_PIGL_compar.par.gamma = 0.13
model_PIGL_compar.find_ss()
model_PIGL_compar.compute_jacs()
model_PIGL_compar.find_transition_path(shocks=shock_PE_s, do_end_check=False)


In [ ]:
paths_ = ['CT', 'CNT', 'CTHF', 'CE', 'CTF', 'CTH']
model.compare_IRFs([model_PIGL_compar, model_CB], ['PIGL', 'Cubb douglas'],  paths_, ncols=3,  T_max=50, do_shocks=False, do_targets=False)

In [ ]:
model_kappa_high = model.copy(name='Baseline with high kappa')
model_kappa_high.par.kappa_w = 0.5
model_kappa_high.find_ss(do_print=False)
model_kappa_high.compute_jacs()


In [ ]:
model_kappa_high.find_transition_path(shocks=shock_PE_s, do_end_check=False)

In [ ]:
model_kappa_high_fixed = model_kappa_high.copy(name='Fixed exchange rate high kappa')
model_kappa_high_fixed.par.float = False
model_kappa_high_fixed.find_ss()
model_kappa_high_fixed.compute_jacs()
model_kappa_high_fixed.find_transition_path(shocks=shock_PE_s, do_end_check=False)


In [ ]:
model_kappa_high.compare_IRFs([model_kappa_high, model_kappa_high_fixed], ['Baseline',  'Fixed exchange'], model.varlist, T_max=50, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
model_PE_i = model.copy(name='Fixed exchange rate')
model_PE_i.find_ss()
model_PE_i.compute_jacs()
model_PE_i.find_transition_path(shocks=shock_PE_i, do_end_check=False)

In [ ]:
plt.plot(shock_PE_i['di_shock'])
plt.plot(shock_PE_i['dPE_s'])

In [ ]:
model_baseline.compare_IRFs([model_baseline, model_PE_i], ['Baseline', 'I shock too'], paths, T_max=500, ncols=3, lvl_value=lvl_path, do_shocks=True, do_targets=False)

In [ ]:
model_baseline.compare_IRFs([model_baseline, model_fixed], ['Baseline', 'Fixed exchange rate'], model.varlist, T_max=100, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
NT_paths = ['YNT', 'WNT', 'inc_NT', 'PNT', 'NNT', 'CNT']
model_baseline.show_IRFs(NT_paths, T_max=50, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
model_high_kappa = model.copy(name='High kappa')
model_high_kappa.par.kappa = 0.3
model_high_kappa.find_ss(do_print=False)
model_high_kappa.compute_jacs()

model_high_kappa.find_transition_path(shocks=shock_PE_s, do_end_check=False)
model_high_kappa.compare_IRFs([model_baseline, model_high_kappa], ['Baseline', 'High kappa'], paths, T_max=50, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
paths_ = ['PE', 'PT', 'CT', 'CNT', 'YH', 'P', 'W', 'w']

In [ ]:
model_baseline.compare_IRFs([model_baseline, model_high_kappa], ['Baseline', 'High kappa'], paths, T_max=50, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
calculations.test_model_properties([model_high_kappa])

In [ ]:
C_list = ['']

In [ ]:
model_high_epsilon = model.copy(name='Low epsilon')
model_high_epsilon.par.epsilon_ = 0.05
model_high_epsilon.find_ss(do_print=False)
model_high_epsilon.compute_jacs()
model_high_epsilon.find_transition_path(shocks=shock_PE_s, do_end_check=False)


In [ ]:
model.compare_IRFs([model_baseline, model_high_epsilon], ['baseline', 'low epsilon'], paths, T_max=50, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
model_gamma_0 = model.copy(name='gamma = 0')
model_gamma_0.par.gamma_ = 0.01
model_gamma_0.find_ss(do_print=False)
model_gamma_0.compute_jacs()
model_gamma_0.find_transition_path(shocks=shock_PE_s, do_end_check=False)


In [ ]:
model_gamma_eq_epsilon = model.copy(name='gamma = epsilon')
model_gamma_eq_epsilon.par.gamma_ = model_gamma_eq_epsilon.par.epsilon_
model_gamma_eq_epsilon.find_ss(do_print=False)
model_gamma_eq_epsilon.compute_jacs()
model_gamma_eq_epsilon.find_transition_path(shocks=shock_PE_s, do_end_check=False)

In [ ]:
model_eta_high = model.copy(name='High eta')
model_eta_high.par.eta_s = 2.0
model_eta_high.par.etaF = 2.0
# model_eta_high.par.etaE = 2.0
# model_eta_high.par.gamma_ = 0.01

model_eta_high.find_ss(do_print=False)
model_eta_high.compute_jacs()
model_eta_high.find_transition_path(shocks=shock_PE_s, do_end_check=False)


In [ ]:
model.compare_IRFs([model_baseline, model_eta_high], ['baseline', 'high_eta'], model.varlist, T_max=50, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
# Expenditure shares and elicticities of substion
for model in [model_baseline,  model_gamma_eq_epsilon, model_gamma_0]:
    print(f'Model: {model.name} \n eta_T = = {model.par.eta_T_RA:.2f}')

In [ ]:
NT_paths = ['YNT', 'WNT', 'inc_NT', 'PNT', 'NNT', 'CNT']
model_gamma_0.compare_IRFs([model_baseline, model_gamma_eq_epsilon, model_gamma_0],['Basline Gamma > epsilon', 'Gamma = epsilon', 'gamma = 0'], model.varlist, T_max=50, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)

for model in [model_baseline, model_gamma_0]:
    a_ = model.ss.a[0,3,:]*model.ss.PNT
    e_ = model.ss.e[0,3,:]*model.ss.PNT
    ect_ = model.ss.ct[0,3,:]*model.ss.PT
    ecnt_ = model.ss.cnt[0,3,:]*model.ss.PNT
    share_ = ect_/(ecnt_+ect_)
    ax.plot(e_, share_, label=model.name)


In [ ]:
shock_PE_s, shock_forigne_interest, shock_PF_s, shock_PF_s_taylor, shock_PE_PF, shock_PE_PF_taylor, shock_i = calculations.difine_shocks(model, scale=0.03, rho = 0.8) 

In [ ]:
paths_r = ['r_real', 'i', 'P', 'ra']
model_baseline.show_IRFs(paths_r, do_shocks=False, do_targets=False, T_max=200, ncols=3, lvl_value=lvl_path)

In [ ]:
YTH = model_baseline.path.YTH
YNT = model_baseline.path.YNT
CNT = model_baseline.path.CNT
CTH = model_baseline.path.CTH 
CTH_s = model_baseline.path.CTH_s

# Differnce form ss 
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1)
ax.plot(YTH - model_baseline.ss.YTH, label='YTH')
ax.plot(YNT - model_baseline.ss.YNT, label='YNT')
ax.plot(CNT - model_baseline.ss.CNT, label='CNT')
ax.plot(CTH - model_baseline.ss.CTH, label='CTH')
ax.plot(CTH_s - model_baseline.ss.CTH_s, label='CTH_s')
ax.legend()

In [ ]:
model_baseline.show_IRFs(model.varlist, do_shocks=False, do_targets=False, T_max=50, ncols=3, lvl_value=lvl_path)

In [ ]:
fig = figs.plot_PE_s(model_baseline)
if save_figs:
    fig.savefig(path + 'PE_s.png', dpi=300)


In [ ]:
fig = figs.plot_jac_p(model_baseline)
if save_figs:
    fig.savefig(path + 'jac_p.png', dpi=300)

In [ ]:

# Expenditure shares 

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(121)

model = model_PIGL_cubdouglaseta


C_list = ['CT', 'CNT']
P_list = ['PT', 'PNT']
# prices = ['P', 'PT', 'PNT', 'PTHF', 'PTH', 'PF'  ]

for i in range(len(C_list)):
    # P_ = P_list[i]
    # P__ = model.path.__dict__[P_]
    dC = (model.path.__dict__[C_list[i]]/model.ss.__dict__[C_list[i]] - 1)*100
    ax.plot(dC[:50],  label = C_list[i])

    dC = (model_CB.path.__dict__[C_list[i]]/model_CB.ss.__dict__[C_list[i]] - 1)*100
    ax.plot(dC[:50],  label = f'CB {C_list[i]}')


# for C in C_list:
#     dC = (model_baseline.path.__dict__[C]/model_baseline.ss.__dict__[C] - 1)*100
#     ax.plot(dC[:50],  label = C)


ax.set_title('Tradable and non-tradable')
ax.legend()

ax = fig.add_subplot(122)

C_list = ['CTHF', 'CE']
P_list = ['PTHF', 'PE']

for i in range(len(C_list)):
    # P_ = P_list[i]
    # P__ = model.path.__dict__[P_]
    # print(C_list[i])
    dC = (model.path.__dict__[C_list[i]]/model.ss.__dict__[C_list[i]] - 1)*100
    ax.plot(dC[:50],  label = C_list[i])

    dC = (model_CB.path.__dict__[C_list[i]]/model_CB.ss.__dict__[C_list[i]] - 1)*100
    ax.plot(dC[:50],  label = f'CB {C_list[i]}')

ax.set_title('Energy and non-energy')
ax.legend()

In [ ]:


fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)

P = 'P'

prices = ['P', 'PT', 'PNT', 'PTHF', 'PTH', 'PF' , 'PE' ]

for P in prices:
    P_ = model_baseline.path.__dict__[P]
    ax.plot(P_[:50],  label = P)
ax.set_title('Price levels')
ax.legend()
plt.show()


In [ ]:
def share_T(par,e, PT=1, PNT=1):
    CT = (e/PT)*(par.nu_*(PNT/e)**par.epsilon_*(PT/PNT)**par.gamma_)
    CNT = (e/PNT)*(1-par.nu_*(PNT/e)**par.epsilon_*(PT/PNT)**par.gamma_)  
    return CT/(CT+CNT) 

In [ ]:
# Marginal propensity to consume/spend
# e = model_baseline.ss.e
par = model_baseline.par


fig = plt.figure()
ax = fig.add_subplot(1,1,1)

i_fix = 0
Nz = 0

for i_z in [1]: #[par.Nz//2-1,par.Nz//2,par.Nz//2+1]:
    
    
    e = model_baseline.ss.e[i_fix,i_z, :]


    t_share = share_T(par,e[1:], PT=model_baseline.ss.PT, PNT=model_baseline.ss.PNT)

    e = model_baseline.ss.e[i_fix,i_z, :]
    MPC = (e[1:]-e[:-1])/((1+model.ss.ra)*(par.a_grid[1:]-par.a_grid[:-1]))
    ax.scatter(t_share, MPC, label=f'Marginal propensity to spend')


    e = model_baseline.ss.ct[i_fix,i_z, :]
    MPC = (e[1:]-e[:-1])/((1+model.ss.ra)*(par.a_grid[1:]-par.a_grid[:-1]))
    ax.scatter(t_share, MPC, label=f'Marginal propensity T')


    e = model_baseline.ss.cnt[i_fix,i_z, :]
    MPC = (e[1:]-e[:-1])/((1+model.ss.ra)*(par.a_grid[1:]-par.a_grid[:-1]))
    ax.scatter(t_share, MPC, label=f'Marginal propensity NT')




#     ax.plot(par.a_grid[:-1],MPC,label=f'z = {par.z_grid[i_z]:.2f}')


ax.set_xlabel('$Tradable share$')

ax.legend(frameon=True);

fig.suptitle('MPC from policy function')
fig.tight_layout(pad=0.5)

# fig.savefig('figs/MPC_policy.pdf')

In [ ]:
# Marginal propensity to consume/spend
e = model_baseline.ss.e
par = model_baseline.par

# MPC = (e[i_fix,Nz, 1:]-e[i_fix, Nz,:-1])/((1+model.ss.ra)*(par.a_grid[1:]-par.a_grid[:-1]))


fig = plt.figure()
ax = fig.add_subplot(1,1,1)

# ax.plot(e[1:], MPC, label='MPC')
# ax.plot(e[1:], t_share, label='Tradable share')

i_fix = 0
Nz = 0

# a_grid initial assets in


for i_z in [1]: #[par.Nz//2-1,par.Nz//2,par.Nz//2+1]:
    e = model_baseline.ss.e[i_fix,i_z, :]
    MPC = (e[1:]-e[:-1])/((1+model.ss.ra)*(par.a_grid[1:]-par.a_grid[:-1]))
    t_share = share_T(par,e[1:], PT=model_baseline.ss.PT, PNT=model_baseline.ss.PNT)
    ax.plot(t_share, MPC, label=f'z = {par.z_grid[i_z]:.2f}')

    


#     ax.plot(par.a_grid[:-1],MPC,label=f'z = {par.z_grid[i_z]:.2f}')


ax.set_xlabel('$Tradable share$')

ax.legend(frameon=True);

fig.suptitle('MPC from policy function')
fig.tight_layout(pad=0.5)

# fig.savefig('figs/MPC_policy.pdf')

In [ ]:
model_high_epsilon = model.copy(name='High epsilon')
model_high_epsilon.par.epsilon_ = model.par.epsilon_ + 0.1
model_high_epsilon.find_ss(do_print=False)
model_high_epsilon.compute_jacs()

In [ ]:
model_high_gamma = model.copy(name='Gamma 0')
model_high_gamma.par.gamma_ = 0.0
model_high_gamma.find_ss(do_print=False)
model_high_gamma.compute_jacs()


In [ ]:
fig = figs.plot_jac_p(model_baseline)


In [ ]:
fig = figs.plot_jac_p(model_high_epsilon)

In [ ]:
fig = figs.plot_jac_p(model_high_gamma)

In [ ]:
# Expenditure on tradable and non-tradable goods as a function of total expenditure
# run_test = False


if run_test:
    a_ = model.ss.a[0,3,:]*model.ss.PNT
    e_ = model.ss.e[0,3,:]*model.ss.PNT
    ect_ = model.ss.ct[0,3,:]*model.ss.PT
    ecnt_ = model.ss.cnt[0,3,:]*model.ss.PNT
    share_ = ect_/(ecnt_+ect_)
    

    plt.figure()
    plt.plot(e_,share_)
    plt.xlabel('Total expenditure')
    plt.ylabel('Expenditure share tradable goods')
    plt.title('Non-homothetic expenditure shares')
    # model.test_hh_path()
    # model.test_path()
    # model.test_jacs()




In [ ]:
model_fixed = model.copy(name='Fixed exchange rate')
# model_fixed.par.nu = 2.0
model_fixed.par.float = False
model_fixed.find_ss(do_print=False)
model_fixed.compute_jacs()
model_fixed.find_transition_path(shocks=shock_PE_s, do_end_check=False)
# model_fixed.show_IRFs(paths, T_max=500, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
models_list = [ model_baseline, model_fixed]
labels = []
for i in models_list:
    labels.append(i.name)
print(labels)

In [ ]:
model_baseline.compare_IRFs(models=models_list, labels=labels, varnames=model.varlist,  T_max=500, ncols=3, lvl_value=lvl_path, do_shocks=True, do_targets=False)

In [ ]:
model_float.compare_IRFs(models_list, labels, ['PT', 'PNT', 'p'], T_max=100, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)

In [ ]:
model_float.compare_IRFs(models_list, labels, model.varlist, T_max=500, ncols=3, lvl_value=lvl_path, do_shocks=False, do_targets=False)